In [171]:
import json
from functools import partial
from inspect import getsource
from math import log
from sys import argv
from bisect import bisect_left
import re

def consecu_int_run_length(idx):
    mark = []
    flag = False
    temp = [ 0, 0, 0 ]
    for i in range(len(k)):
        if i != idx[i]:
            if flag == False:
                temp = [ i, idx[i], 1 ]
                flag = True
            elif flag == True and idx[i]-(temp[1]+temp[2]) == 0:
                temp[2] += 1
            elif flag == True:
                mark += temp
                temp = [ i, idx[i], 1 ]
                flag = True
        else:
            if flag == True:
                mark += temp
                flag = False
    return mark
if __name__ == '__main__':
    map_in_name, script_name = '../cases/case1.json', 'test.py'
    nf = '../input/case1_in.json'
    nf = open(nf, 'r')
    nf = json.load(nf)
    keys, values = sorted(nf[0]), sorted(nf[1])
    
    map_in_file = open(map_in_name)
    origin_mapping = json.load(map_in_file)
    map_in_file.close()

    mmm = sorted(list(origin_mapping.items()),key=lambda x: x[0])

    k,v = zip(*mmm)
    mm = sorted(zip(v,[ i for i in range(len(k)) ]),key=lambda x: x[0])
    v, idx = zip(*mm)
    idx = list(idx)
    
    mark = consecu_int_run_length(idx)
 
    #print(len(str(mark)))
    om = []
    for i in range(0,len(mark),3):
        om.append(mark[i:i+3])
        print(mark[i:i+3])
    
    #mark_file = open('mark/'+re.search('case[0-9]+',str(map_in_file)).group(0)+'.mark', 'w')
    #mark_file.write(str(om))
    #mark_file.close()
    
    document = """
import json
import hashlib
from sys import argv

_m = {}
nf, of = argv[1], argv[2]
nf = open(nf, 'r')
nf = json.load(nf)
keys, values = sorted(nf[0]), sorted(nf[1])

nm = {{}}
for i in range(len(keys)):
    nm[keys[i]] = values[i]
for i in range(len(_m)):
    for j in range(_m[i][2]):
        nm[keys[_m[i][1]+j]] = values[_m[i][0]+j]
of = open(of, 'w')
of.write(json.dumps(nm))
of.close()
    """.format(str(mark))
    script = open(script_name, 'w')
    script.write(document)
    script.close()

[3, 16, 2]
[5, 6, 4]
[9, 3, 3]
[12, 10, 6]
[19, 32, 2]
[21, 22, 4]
[25, 19, 3]
[28, 26, 6]
[35, 36, 6]
[42, 6, 0]
[51, 64, 2]
[53, 54, 4]
[57, 51, 3]
[60, 58, 6]
[67, 80, 2]
[69, 70, 4]
[73, 67, 3]
[76, 74, 6]
[83, 96, 2]
[85, 86, 4]
[89, 83, 3]
[92, 90, 6]
[99, 112, 2]
[101, 102, 4]
[105, 99, 3]
[108, 106, 6]
[116, -3, 0]
[116, 120, 4]
[121, 1, 0]
[121, 115, 3]
[131, 144, 2]
[133, 134, 4]
[137, 131, 3]
[140, 138, 6]
[147, 160, 2]
[149, 150, 4]
[153, 147, 3]
[156, 154, 6]
[163, 176, 2]
[165, 166, 4]
[169, 163, 3]
[172, 170, 6]
[179, 192, 2]
[181, 182, 4]
[185, 179, 3]
[188, 186, 6]
[195, 208, 2]
[197, 198, 4]
[201, 195, 3]
[204, 202, 6]
[211, 224, 2]
[213, 214, 4]
[217, 211, 3]
[220, 218, 6]
[227, 240, 2]
[229, 230, 4]
[233, 227, 3]
[236, 234, 6]
[243, 256, 2]
[245, 246, 4]
[249, 243, 3]
[252, 250, 6]
[259, 272, 2]
[261, 262, 4]
[265, 259, 3]
[268, 266, 6]
[275, 288, 2]
[277, 278, 4]
[281, 275, 3]
[284, 282, 6]
[291, 304, 2]
[293, 294, 4]
[297, 291, 3]
[300, 298, 6]
[307, 320, 2]
[310,